In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC, SVR
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, cohen_kappa_score, accuracy_score,make_scorer 

In [3]:
# !pip install shap
import shap

In [4]:
path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/ozone/eighthr.data'
data = pd.read_csv(path, header=None,na_values=['?','##'])
# data = pd.read_csv(path, header=None,na_values='?')
data.head()

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,1/1/1998,0.8,1.8,2.4,2.1,2.0,2.1,1.5,1.7,1.9,...,0.15,10.67,-1.56,5795.0,-12.10,17.9,10330.0,-55.0,0.00,0.0
1,1/2/1998,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,...,0.48,8.39,3.84,5805.0,14.05,29.0,10275.0,-55.0,0.00,0.0
2,1/3/1998,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,...,0.60,6.94,9.80,5790.0,17.90,41.3,10235.0,-40.0,0.00,0.0
3,1/4/1998,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,...,0.49,8.73,10.54,5775.0,31.15,51.7,10195.0,-40.0,2.08,0.0
4,1/5/1998,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.58,0.0


In [5]:
data.shape

(2534, 74)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2534 entries, 0 to 2533
Data columns (total 74 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2534 non-null   object 
 1   1       2235 non-null   float64
 2   2       2242 non-null   float64
 3   3       2240 non-null   float64
 4   4       2242 non-null   float64
 5   5       2241 non-null   float64
 6   6       2242 non-null   float64
 7   7       2243 non-null   float64
 8   8       2245 non-null   float64
 9   9       2244 non-null   float64
 10  10      2247 non-null   float64
 11  11      2246 non-null   float64
 12  12      2242 non-null   float64
 13  13      2247 non-null   float64
 14  14      2246 non-null   float64
 15  15      2246 non-null   float64
 16  16      2248 non-null   float64
 17  17      2250 non-null   float64
 18  18      2251 non-null   float64
 19  19      2248 non-null   float64
 20  20      2242 non-null   float64
 21  21      2240 non-null   float64
 22  

In [7]:
var_name_path ='https://archive.ics.uci.edu/ml/machine-learning-databases/ozone/eighthr.names'
from urllib.request import urlopen
f = urlopen(var_name_path).read()
f

b'1,0 | two classes 1: ozone day, 0: normal day\r\n\r\nDate:     ignore.\r\nWSR0:     continuous.\r\nWSR1:     continuous.\r\nWSR2:     continuous.\r\nWSR3:     continuous.\r\nWSR4:     continuous.\r\nWSR5:     continuous.\r\nWSR6:     continuous.\r\nWSR7:     continuous.\r\nWSR8:     continuous.\r\nWSR9:     continuous.\r\nWSR10:     continuous.\r\nWSR11:     continuous.\r\nWSR12:     continuous.\r\nWSR13:     continuous.\r\nWSR14:     continuous.\r\nWSR15:     continuous.\r\nWSR16:     continuous.\r\nWSR17:     continuous.\r\nWSR18:     continuous.\r\nWSR19:     continuous.\r\nWSR20:     continuous.\r\nWSR21:     continuous.\r\nWSR22:     continuous.\r\nWSR23:     continuous.\r\nWSR_PK:     continuous.\r\nWSR_AV:     continuous.\r\nT0:     continuous.\r\nT1:     continuous.\r\nT2:     continuous.\r\nT3:     continuous.\r\nT4:     continuous.\r\nT5:     continuous.\r\nT6:     continuous.\r\nT7:     continuous.\r\nT8:     continuous.\r\nT9:     continuous.\r\nT10:     continuous.\r\nT1

In [8]:
import re

In [9]:
starting_point = re.search('Date',str(f))
starting_point.span()

(55, 59)

In [10]:
starting_point

<re.Match object; span=(55, 59), match='Date'>

In [11]:
f1 = str(f)[starting_point.span()[0]:]  # starting from 55 till the end of f
f1

"Date:     ignore.\\r\\nWSR0:     continuous.\\r\\nWSR1:     continuous.\\r\\nWSR2:     continuous.\\r\\nWSR3:     continuous.\\r\\nWSR4:     continuous.\\r\\nWSR5:     continuous.\\r\\nWSR6:     continuous.\\r\\nWSR7:     continuous.\\r\\nWSR8:     continuous.\\r\\nWSR9:     continuous.\\r\\nWSR10:     continuous.\\r\\nWSR11:     continuous.\\r\\nWSR12:     continuous.\\r\\nWSR13:     continuous.\\r\\nWSR14:     continuous.\\r\\nWSR15:     continuous.\\r\\nWSR16:     continuous.\\r\\nWSR17:     continuous.\\r\\nWSR18:     continuous.\\r\\nWSR19:     continuous.\\r\\nWSR20:     continuous.\\r\\nWSR21:     continuous.\\r\\nWSR22:     continuous.\\r\\nWSR23:     continuous.\\r\\nWSR_PK:     continuous.\\r\\nWSR_AV:     continuous.\\r\\nT0:     continuous.\\r\\nT1:     continuous.\\r\\nT2:     continuous.\\r\\nT3:     continuous.\\r\\nT4:     continuous.\\r\\nT5:     continuous.\\r\\nT6:     continuous.\\r\\nT7:     continuous.\\r\\nT8:     continuous.\\r\\nT9:     continuous.\\r\\nT10:  

In [12]:
var_names = re.findall("[A-Za-z0-9_]+(?=:)",f1)
print(len(var_names))
var_names.append('Class')
print(var_names)

73
['Date', 'nWSR0', 'nWSR1', 'nWSR2', 'nWSR3', 'nWSR4', 'nWSR5', 'nWSR6', 'nWSR7', 'nWSR8', 'nWSR9', 'nWSR10', 'nWSR11', 'nWSR12', 'nWSR13', 'nWSR14', 'nWSR15', 'nWSR16', 'nWSR17', 'nWSR18', 'nWSR19', 'nWSR20', 'nWSR21', 'nWSR22', 'nWSR23', 'nWSR_PK', 'nWSR_AV', 'nT0', 'nT1', 'nT2', 'nT3', 'nT4', 'nT5', 'nT6', 'nT7', 'nT8', 'nT9', 'nT10', 'nT11', 'nT12', 'nT13', 'nT14', 'nT15', 'nT16', 'nT17', 'nT18', 'nT19', 'nT20', 'nT21', 'nT22', 'nT23', 'nT_PK', 'nT_AV', 'nT85', 'nRH85', 'nU85', 'nV85', 'nHT85', 'nT70', 'nRH70', 'nU70', 'nV70', 'nHT70', 'nT50', 'nRH50', 'nU50', 'nV50', 'nHT50', 'nKI', 'nTT', 'nSLP', 'nSLP_', 'nPrecp', 'Class']


In [13]:
len(var_names)

74

In [14]:
data.columns = var_names
data.head()

,Date,nWSR0,nWSR1,nWSR2,nWSR3,nWSR4,nWSR5,nWSR6,nWSR7,nWSR8,...,nRH50,nU50,nV50,nHT50,nKI,nTT,nSLP,nSLP_,nPrecp,Class
0,1/1/1998,0.8,1.8,2.4,2.1,2.0,2.1,1.5,1.7,1.9,...,0.15,10.67,-1.56,5795.0,-12.10,17.9,10330.0,-55.0,0.00,0.0
1,1/2/1998,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,...,0.48,8.39,3.84,5805.0,14.05,29.0,10275.0,-55.0,0.00,0.0
2,1/3/1998,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,...,0.60,6.94,9.80,5790.0,17.90,41.3,10235.0,-40.0,0.00,0.0
3,1/4/1998,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,...,0.49,8.73,10.54,5775.0,31.15,51.7,10195.0,-40.0,2.08,0.0
4,1/5/1998,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.58,0.0


In [15]:
data.Class.value_counts()

0.0    2374
1.0     160
Name: Class, dtype: int64

In [16]:
data.isnull().sum()

Date        0
nWSR0     299
nWSR1     292
nWSR2     294
nWSR3     292
         ... 
nTT       125
nSLP       95
nSLP_     158
nPrecp      2
Class       0
Length: 74, dtype: int64

In [17]:
data.isnull().sum().sum()

14937

In [18]:
1 - len(data.dropna())/len(data)  # % of information loss

0.271112865035517

In [19]:
# missing value imputation

In [20]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import GradientBoostingRegressor as GBR, AdaBoostRegressor as ABR

In [21]:
imputer = IterativeImputer(estimator=ABR(),max_iter=20)  # google mice

In [22]:
x_train,x_test, y_train, y_test=train_test_split(data.iloc[:,1:73],
                                                data.iloc[:,73],
                                                test_size=0.3,
                                                random_state=12345)

In [23]:
x_train_complete = imputer.fit_transform(x_train)
x_test_complete = imputer.transform(x_test)

[IterativeImputer] Early stopping criterion not reached.


In [24]:
x_train_complete

array([[ 1.8000e+00,  1.3000e+00,  1.3000e+00, ...,  1.0135e+04,
        -2.0000e+01,  0.0000e+00],
       [ 1.1000e+00,  6.0000e-01,  6.0000e-01, ...,  1.0260e+04,
        -5.5000e+01,  0.0000e+00],
       [ 2.6000e+00,  2.5000e+00,  1.5000e+00, ...,  1.0155e+04,
         1.5000e+01,  0.0000e+00],
       ...,
       [ 6.2000e+00,  6.8000e+00,  5.4000e+00, ...,  1.0070e+04,
         4.5000e+01,  0.0000e+00],
       [ 1.4000e+00,  1.4000e+00,  1.1000e+00, ...,  1.0270e+04,
         6.0000e+01,  0.0000e+00],
       [ 2.4000e+00,  2.5000e+00,  3.2000e+00, ...,  1.0140e+04,
        -1.0000e+01,  0.0000e+00]])

In [25]:
df_train = pd.DataFrame(data=x_train_complete,columns=data.columns[1:73])

In [26]:
df_test = pd.DataFrame(data=x_test_complete,columns=data.columns[1:73])

In [27]:
df_train.shape, df_test.shape

((1773, 72), (761, 72))

In [28]:
from sklearn.preprocessing import StandardScaler

In [29]:
sc = StandardScaler()

In [30]:
x_train_complete_norm = sc.fit_transform(x_train_complete)
x_test_complete_norm = sc.transform(x_test_complete)

In [31]:
svm = SVC()

In [32]:
svm_param = {'C':[1.0,10.0,20.0,50.0,75],
             'gamma':[0.0001, 0.001,0.01,0.1],
            'kernel':['liner','poly','rbf','sigmoid'],
            }

In [33]:
def kappa(y_act, y_pred):
    return cohen_kappa_score(y_act, y_pred)
Kappa = make_scorer(kappa)

In [34]:
best_svm = RandomizedSearchCV(estimator=svm,
                              param_distributions=svm_param,
                             n_iter=100,
                             n_jobs=-1,
                             scoring=Kappa, cv=5,random_state=123456)

In [35]:
best_svm.fit(X=x_train_complete_norm,y=y_train)

The total space of parameters 80 is smaller than n_iter=100. Running 80 iterations. For exhaustive searches, use GridSearchCV.

100 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SHAHRUKH\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SHAHRUKH\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\SHAHRUKH\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=100, n_jobs=-1,
                   param_distributions={'C': [1.0, 10.0, 20.0, 50.0, 75],
                                        'gamma': [0.0001, 0.001, 0.01, 0.1],
                                        'kernel': ['liner', 'poly', 'rbf',
                                                   'sigmoid']},
                   random_state=123456, scoring=make_scorer(kappa))

In [36]:
best_svm.best_params_

{'kernel': 'poly', 'gamma': 0.01, 'C': 50.0}

In [37]:
pred_svm = best_svm.best_estimator_.predict(x_test_complete_norm)

In [38]:
confusion_matrix(y_test, pred_svm)

array([[690,  18],
       [ 36,  17]], dtype=int64)

In [39]:
pred_svm

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.